### Candes and Recht algorithm

`M: The true matrix.
n_1: The number of row of the matrix M.
n_2: The number of column of the matrix M.
r: The rank of the matrix M.
omega: The sample set. observed entry (i, j) in the sample set omega.
M_: Input of algorithm. The matrix to recover.
X_: The matrix to recover in SDP.
X: The solution of nuclear norm minimization.`

## Importing required libraries

In [1]:
import random
import math
import itertools
import time
import random
import numpy as np
from cvxpy import *
k_dict = {}

## Algorithm

In [3]:
# Noting starting time
start = time.time()
random.seed(0)
rank, p = 5, 5


#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
def _solve(M, omega):
    n_1 = M.shape[0]
    n_2 = M.shape[1]
    X_ = Semidef(n_1 + n_2)
    objective = Minimize(trace(X_))

    constraints = [(X_ == X_.T)]  # add symmetric constraint.
    for i, j in omega:
        constr = (X_[i, j + n_1] == M[i, j])
        constraints.append(constr)
    problem = Problem(objective, constraints)
    problem.solve(solver=CVXOPT)

    print("STATUS       :", problem.status)
    print("OPTIMAL VALUE:", problem.value)

    X0 = X_.value
    # check optimizer's solution is symmetric.
    print("|X0-X0.T|_F  :", np.linalg.norm(np.subtract(X0, X0.T), "fro"))
    return X_.value[:n_1, n_1:]

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

def _generate_omega(n_1, n_2, m):
    random.seed(0)
    return random.sample([(i, j) for i in range(n_1-1) for j in range(n_2-1)], m)


def _get_mask_matrix(n_1, n_2, omega):
    """
    If we observed entry (i, j) of matrix M, the entry of mask matrix is 1,
    Otherwise 0.
    """
    mask = np.zeros((n_1, n_2), dtype=np.int8)
    for i, j in omega:
        mask[i, j] = 1
    return mask


def _get_abs_max_from_matrix(M):
    return np.max(np.absolute(M))

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
def main(n_1, n_2, r, m):
    print("#row of M    :", n_1)
    print("#column of M :", n_2)
    print("#sample      :", m)
    random.seed(0)
    L = np.array(np.random.randint(p,size=(row_order,rank)),dtype=float)
    R = np.array(np.random.randint(p,size=(col_order,rank)),dtype=float)
    M = L.dot(R.T)%p
    M_abs_max = _get_abs_max_from_matrix(M)
    # print("RANK of M    :", np.linalg.matrix_rank(M))
    print("|M|_*        :", np.linalg.norm(M, "nuc"))
    M_rank = np.linalg.matrix_rank(M)
    print("RANK of M    :", M_rank)
    U, S, V_T = np.linalg.svd(M, full_matrices=False)
    print(S)

    omega = _generate_omega(n_1, n_2, m)
    
    #for calculating K
    i,_ = zip(*omega)
    
    mask = _get_mask_matrix(n_1, n_2, omega)

    # M_ is for training data removing the data.
    # This block should not affect the result.
    # Just for defensive programming.
    # M_ = np.random.uniform(-M_abs_max, M_abs_max, M.shape)
    M_ = M.copy()
    np.place(M_, 1 - mask, M_abs_max * M_abs_max)
    X = _solve(M_, omega)
    X_rank = np.linalg.matrix_rank(X)
    print("RANK of X    :", X_rank)
    print("|X|_*        :", np.linalg.norm(X, "nuc"))
    U, S, V_T = np.linalg.svd(X, full_matrices=False)
    print(S)

    E = np.subtract(M, X)
    E_train = E.copy()
    np.place(E_train, 1 - mask, 0)
    print('TRAIN RMSE   :', np.linalg.norm(E_train, "fro") / m)
    E_test = E.copy()
    np.place(E_test, mask, 0)
    print('TEST  RMSE   :', np.linalg.norm(E_test, "fro") / (n_1 * n_2 - m))

    frobenius_norm_ratio = (
        np.linalg.norm(X - M, "fro") / np.linalg.norm(M, "fro"))
    print("|X-M|_F/|M|_F:", frobenius_norm_ratio)  # the metric of paper.

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
if __name__ == '__main__':
    row_order,col_order = (50,30)
    q = 0.4
    main(row_order,col_order, r=2, m=int(row_order * col_order * q))

#row of M    : 50
#column of M : 30
#sample      : 600
|M|_*        : 343.5895610946623
RANK of M    : 30
[79.37476038 16.78679047 15.63817715 14.87347902 14.04660342 13.29268872
 13.09834468 12.5367474  12.0600272  11.60430432 11.15630899 10.9591045
 10.46364893  9.91103125  9.64083746  8.93049595  8.46673553  8.25580114
  7.90585076  7.28195346  6.83633487  6.41226258  5.34605876  4.95817168
  4.84037997  4.68116214  4.21285944  3.70405958  3.39825395  2.91632738]
STATUS       : optimal
OPTIMAL VALUE: 419.3497008673506
|X0-X0.T|_F  : 0.0
RANK of X    : 29
|X|_*        : 209.674850450624
[6.88529237e+01 1.78355782e+01 1.56492160e+01 1.38855989e+01
 1.25637484e+01 1.15899788e+01 1.11654361e+01 9.92746847e+00
 8.83172650e+00 7.27472772e+00 6.29859996e+00 5.77872269e+00
 5.07047378e+00 4.45237896e+00 4.13944768e+00 2.72310972e+00
 1.89349960e+00 1.33446391e+00 4.07751362e-01 5.78715172e-09
 3.38682681e-09 3.04112134e-09 1.85503294e-09 1.31909843e-09
 1.06753856e-09 9.53332757e-10 7.62228